In [ ]:

# %%
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configurar seed para reproducibilidad
np.random.seed(42)



In [ ]:

# Cargar el dataset de vinos tintos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine_data = pd.read_csv(url, sep=';')

print("Dataset cargado exitosamente!")
print(f"Dimensiones del dataset: {wine_data.shape}")
print(f"\nColumnas del dataset:")
print(wine_data.columns.tolist())



In [ ]:

# ## 2. Análisis Exploratorio de Datos (EDA)
#
# ### Ejercicio 2.1: Exploración inicial
# Completa el análisis exploratorio inicial del dataset.



In [ ]:

# Mostrar las primeras filas del dataset
wine_data.head()



In [ ]:

# TODO: Muestra la información general del dataset (tipos de datos, valores no nulos)
# Tu código aquí
wine_data.info()



In [ ]:

# TODO: Calcula y muestra las estadísticas descriptivas del dataset
# Tu código aquí
wine_data.describe().T



In [ ]:

# TODO: Verifica si hay valores nulos en el dataset
# Tu código aquí
wine_data.isna().sum()



In [ ]:

# ### Ejercicio 2.2: Análisis de la variable objetivo



In [ ]:

# Analizar la distribución de la calidad del vino
plt.figure(figsize=(10, 6))
wine_data['quality'].value_counts().sort_index().plot(kind='bar', color='steelblue', edgecolor='black')
plt.xlabel('Calidad del Vino', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Distribución de la Calidad del Vino', fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

# Agregar estadísticas
mean_quality = wine_data['quality'].mean()
median_quality = wine_data['quality'].median()
plt.axhline(y=wine_data['quality'].value_counts().mean(), color='red',
            linestyle='--', label=f'Media de frecuencia')

plt.legend()
plt.tight_layout()
plt.show()

print(f"Estadísticas de la calidad del vino:")
print(f"Media: {mean_quality:.2f}")
print(f"Mediana: {median_quality:.2f}")
print(f"Desviación estándar: {wine_data['quality'].std():.2f}")


In [ ]:

# ### Ejercicio 2.3: Matriz de correlación


# TODO: Calcula la matriz de correlación y visualízala con un heatmap
# Pista: Usa sns.heatmap() con annot=True para mostrar los valores
# Tu código aquí
plt.figure(figsize=(14, 10))
corr = wine_data.corr(numeric_only=True)
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=False, cbar=True)
plt.title("Matriz de correlación")
plt.tight_layout()
plt.show()



In [ ]:

# TODO: Identifica y muestra las 5 variables más correlacionadas con 'quality'
# Tu código aquí
target_corr = corr['quality'].drop('quality').sort_values(ascending=False)
top5_corr = target_corr.head(5)
print("Top 5 variables más correlacionadas positivamente con 'quality':")
print(top5_corr)
print("\nTop 5 variables más correlacionadas (en valor absoluto):")
print(target_corr.abs().sort_values(ascending=False).head(5))



In [ ]:

# ### Ejercicio 2.4: Visualización de relaciones

# Visualizar las 4 variables más correlacionadas con quality
top_features = ['alcohol', 'volatile acidity', 'citric acid', 'sulphates']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Relación entre Variables Principales y Calidad del Vino', fontsize=16)

for ax, feature in zip(axes.flat, top_features):
    # TODO: Crea un scatter plot para cada variable vs quality
    # Agrega una línea de tendencia
    # Tu código aquí
    x = wine_data[feature]
    y = wine_data['quality']
    ax.scatter(x, y, alpha=0.4, edgecolor='k')
    # línea de tendencia
    m, b = np.polyfit(x, y, 1)
    x_line = np.linspace(x.min(), x.max(), 100)
    ax.plot(x_line, m*x_line + b, linestyle='--')
    ax.set_xlabel(feature)
    ax.set_ylabel('quality')
    ax.set_title(f'{feature} vs quality (pendiente {m:.2f})')

plt.tight_layout()
plt.show()



In [ ]:

# ## 3. Preparación de Datos

# Separar características (X) y variable objetivo (y)
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"\nCaracterísticas: {X.columns.tolist()}")


In [ ]:


# TODO: Divide los datos en conjuntos de entrenamiento y prueba
# Usa test_size=0.2 y random_state=42
# Tu código aquí

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")



In [ ]:

# TODO: Estandariza las características
# Recuerda: ajusta el scaler solo con los datos de entrenamiento
# Tu código aquí

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:

# ## 4. Validación Cruzada para Selección de Hiperparámetros
#
# La validación cruzada es fundamental para seleccionar los mejores hiperparámetros sin usar el conjunto de prueba.


# ### Ejercicio 4.1: Implementación manual de validación cruzada


# Ejemplo: Validación cruzada manual para Ridge
from sklearn.linear_model import Ridge

def manual_cross_validation(X, y, alpha, n_folds=5):
    """
    Implementa validación cruzada manualmente para Ridge regression
    """
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in kf.split(X):
        # Dividir datos
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
        
        # Estandarizar
        scaler_cv = StandardScaler()
        X_train_cv_scaled = scaler_cv.fit_transform(X_train_cv)
        X_val_cv_scaled = scaler_cv.transform(X_val_cv)
        
        # Entrenar modelo
        model = Ridge(alpha=alpha, random_state=42)
        model.fit(X_train_cv_scaled, y_train_cv)
        
        # Evaluar
        y_pred = model.predict(X_val_cv_scaled)
        rmse = np.sqrt(mean_squared_error(y_val_cv, y_pred))
        scores.append(rmse)
    
    return np.mean(scores), np.std(scores)

# Probar diferentes valores de alpha
alphas_to_test = [0.001, 0.01, 0.1, 1, 10, 100]
cv_results_manual = []

print("Validación Cruzada Manual para Ridge Regression:")
print("-" * 50)

for alpha in alphas_to_test:
    mean_rmse, std_rmse = manual_cross_validation(X.values, y, alpha)
    cv_results_manual.append({'alpha': alpha, 'mean_rmse': mean_rmse, 'std_rmse': std_rmse})
    print(f"Alpha: {alpha:7.3f} | RMSE: {mean_rmse:.4f} (+/- {std_rmse:.4f})")

# TODO: Identifica el mejor alpha basado en el RMSE medio más bajo
# Tu código aquí
cv_df = pd.DataFrame(cv_results_manual)
best_alpha_manual = cv_df.loc[cv_df['mean_rmse'].idxmin(), 'alpha']
print(f"\nMejor alpha (CV manual): {best_alpha_manual}")



In [ ]:

# ### Ejercicio 4.2: Usar RidgeCV para validación cruzada automática


# TODO: Usa RidgeCV para encontrar automáticamente el mejor alpha
# Pista: RidgeCV tiene un parámetro 'alphas' y 'cv'
# Tu código aquí

alphas = np.logspace(-3, 3, 100)  # 100 valores entre 0.001 y 1000
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train_scaled, y_train)

print(f"Mejor alpha encontrado por RidgeCV: {ridge_cv.alpha_:.6f}")



In [ ]:

# ### Ejercicio 4.3: GridSearchCV para búsqueda exhaustiva

# Ejemplo completo con GridSearchCV para Ridge
from sklearn.model_selection import GridSearchCV

# Definir parámetros a buscar
param_grid_ridge = {
    'alpha': np.logspace(-3, 3, 20)  # 20 valores entre 0.001 y 1000
}

# Crear modelo base
ridge_base = Ridge(random_state=42)

# TODO: Implementa GridSearchCV
# Usa cv=5, scoring='neg_mean_squared_error'
# Tu código aquí
grid_search_ridge = GridSearchCV(
    estimator=ridge_base,
    param_grid=param_grid_ridge,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    return_train_score=True
)
grid_search_ridge.fit(X_train_scaled, y_train)

print(f"Mejor alpha (GridSearchCV): {grid_search_ridge.best_params_['alpha']}")
print(f"Mejor score (RMSE CV): {np.sqrt(-grid_search_ridge.best_score_):.4f}")



In [ ]:

# TODO: Visualiza los resultados de la validación cruzada
# Crea un gráfico que muestre cómo cambia el RMSE con diferentes valores de alpha
# Tu código aquí

results = grid_search_ridge.cv_results_
alphas_gs = results['param_alpha'].data.astype(float)
rmse_means = np.sqrt(-results['mean_test_score'])
rmse_stds = np.sqrt(-results['std_test_score']**2)  # std ya viene en score, sólo para claridad

plt.figure(figsize=(10, 6))
plt.plot(alphas_gs, rmse_means, marker='o')
plt.xscale('log')
plt.xlabel('alpha (log)')
plt.ylabel('RMSE (CV)')
plt.title('Ridge: RMSE en CV vs alpha')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



In [ ]:

# ## 5. Modelos de Regresión
#
# ### 5.1 Regresión Lineal Normal


# TODO: Implementa y entrena un modelo de regresión lineal
# Tu código aquí

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred_lr_train = lr_model.predict(X_train_scaled)
y_pred_lr_test  = lr_model.predict(X_test_scaled)

# Calcular métricas
rmse_lr_train = np.sqrt(mean_squared_error(y_train, y_pred_lr_train))
rmse_lr_test  = np.sqrt(mean_squared_error(y_test, y_pred_lr_test))
r2_lr_train   = r2_score(y_train, y_pred_lr_train)
r2_lr_test    = r2_score(y_test, y_pred_lr_test)

print("Regresión Lineal Normal:")
print(f"RMSE Train: {rmse_lr_train:.4f}")
print(f"RMSE Test : {rmse_lr_test:.4f}")
print(f"R² Train : {r2_lr_train:.4f}")
print(f"R² Test  : {r2_lr_test:.4f}")



In [ ]:

# ### 5.2 Ridge Regression con mejor alpha de CV


# TODO: Entrena Ridge con el mejor alpha encontrado por validación cruzada
# Tu código aquí

best_alpha_ridge = ridge_cv.alpha_  # de la sección anterior
ridge_model = Ridge(alpha=best_alpha_ridge, random_state=42)
ridge_model.fit(X_train_scaled, y_train)

# Predicciones y métricas
y_pred_ridge_train = ridge_model.predict(X_train_scaled)
y_pred_ridge_test  = ridge_model.predict(X_test_scaled)

rmse_ridge_train = np.sqrt(mean_squared_error(y_train, y_pred_ridge_train))
rmse_ridge_test  = np.sqrt(mean_squared_error(y_test, y_pred_ridge_test))
r2_ridge_train   = r2_score(y_train, y_pred_ridge_train)
r2_ridge_test    = r2_score(y_test, y_pred_ridge_test)

print("\nRidge Regression:")
print(f"alpha óptimo: {best_alpha_ridge:.6f}")
print(f"RMSE Train: {rmse_ridge_train:.4f}")
print(f"RMSE Test : {rmse_ridge_test:.4f}")
print(f"R² Train : {r2_ridge_train:.4f}")
print(f"R² Test  : {r2_ridge_test:.4f}")



In [ ]:

# ### 5.3 Lasso Regression con validación cruzada

# TODO: Implementa LassoCV para encontrar el mejor alpha automáticamente
# Tu código aquí

alphas_lasso = np.logspace(-3, 1, 100)
lasso_cv = LassoCV(alphas=alphas_lasso, cv=5, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)

print(f"\nMejor alpha para Lasso: {lasso_cv.alpha_:.6f}")
print(f"Número de características seleccionadas: {(lasso_cv.coef_ != 0).sum()} / {X.shape[1]}")

# Predicciones y métricas
y_pred_lasso_train = lasso_cv.predict(X_train_scaled)
y_pred_lasso_test  = lasso_cv.predict(X_test_scaled)

rmse_lasso_train = np.sqrt(mean_squared_error(y_train, y_pred_lasso_train))
rmse_lasso_test  = np.sqrt(mean_squared_error(y_test, y_pred_lasso_test))
r2_lasso_train   = r2_score(y_train, y_pred_lasso_train)
r2_lasso_test    = r2_score(y_test, y_pred_lasso_test)

print("\nLasso Regression (CV):")
print(f"RMSE Train: {rmse_lasso_train:.4f}")
print(f"RMSE Test : {rmse_lasso_test:.4f}")
print(f"R² Train : {r2_lasso_train:.4f}")
print(f"R² Test  : {r2_lasso_test:.4f}")



In [ ]:

# TODO: Identifica qué características fueron eliminadas por Lasso
# Tu código aquí
lasso_coefs = pd.Series(lasso_cv.coef_, index=X.columns)
dropped_features = lasso_coefs[lasso_coefs == 0].index.tolist()
print("\nCaracterísticas eliminadas por Lasso:")
print(dropped_features)



In [ ]:

# ## 6. Comparación de Modelos

# TODO: Crea una tabla comparativa con todos los modelos
# Incluye: RMSE Train, RMSE Test, R² Train, R² Test, MAE Test
# Tu código aquí

comparison_data = {
    'Modelo':      ['Linear', 'Ridge', 'Lasso'],
    'RMSE Train':  [rmse_lr_train, rmse_ridge_train, rmse_lasso_train],
    'RMSE Test':   [rmse_lr_test,  rmse_ridge_test,  rmse_lasso_test],
    'R² Train':    [r2_lr_train,   r2_ridge_train,   r2_lasso_train],
    'R² Test':     [r2_lr_test,    r2_ridge_test,    r2_lasso_test],
    'MAE Test':    [
        mean_absolute_error(y_test, y_pred_lr_test),
        mean_absolute_error(y_test, y_pred_ridge_test),
        mean_absolute_error(y_test, y_pred_lasso_test)
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\nComparación de modelos:")
print(comparison_df)



In [ ]:

# TODO: Crea visualizaciones para comparar los modelos
# 1. Gráfico de barras comparando RMSE
# 2. Gráfico de barras comparando R²
# Tu código aquí

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.barplot(x='Modelo', y='RMSE Test', data=comparison_df, ax=ax[0])
ax[0].set_title('Comparación RMSE (Test)')
ax[0].set_ylabel('RMSE')

sns.barplot(x='Modelo', y='R² Test', data=comparison_df, ax=ax[1])
ax[1].set_title('Comparación R² (Test)')
ax[1].set_ylabel('R²')

plt.tight_layout()
plt.show()



In [ ]:

# ## 7. Análisis de Residuos


# TODO: Para el mejor modelo, crea:
# 1. Gráfico de residuos vs predicciones
# 2. Histograma de residuos
# 3. Q-Q plot de residuos
# Tu código aquí

# Elegimos el mejor por menor RMSE Test
best_idx = comparison_df['RMSE Test'].idxmin()
best_model_name = comparison_df.loc[best_idx, 'Modelo']

if best_model_name == 'Linear':
    best_model = lr_model
    y_pred_best = y_pred_lr_test
elif best_model_name == 'Ridge':
    best_model = ridge_model
    y_pred_best = y_pred_ridge_test
else:
    best_model = lasso_cv
    y_pred_best = y_pred_lasso_test

residuals = y_test - y_pred_best

# 1) Residuos vs predicciones
plt.figure(figsize=(7,5))
plt.scatter(y_pred_best, residuals, alpha=0.5, edgecolor='k')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicciones')
plt.ylabel('Residuos')
plt.title(f'Residuos vs Predicciones ({best_model_name})')
plt.tight_layout()
plt.show()

# 2) Histograma de residuos
plt.figure(figsize=(7,5))
plt.hist(residuals, bins=30, edgecolor='k')
plt.title('Histograma de Residuos')
plt.xlabel('Residuo')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

# 3) Q-Q plot
import scipy.stats as stats
plt.figure(figsize=(7,5))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot de Residuos')
plt.tight_layout()
plt.show()



In [ ]:

# ## 8. Importancia de Características


# TODO: Visualiza los coeficientes de los tres modelos en un mismo gráfico
# Esto te ayudará a entender qué características son más importantes
# Tu código aquí

coef_linear = pd.Series(lr_model.coef_, index=X.columns)
coef_ridge  = pd.Series(ridge_model.coef_, index=X.columns)
coef_lasso  = pd.Series(lasso_cv.coef_, index=X.columns)

coef_df = pd.DataFrame({
    'Linear': coef_linear,
    'Ridge':  coef_ridge,
    'Lasso':  coef_lasso
})

coef_df.plot(kind='bar', figsize=(14,6))
plt.title('Coeficientes por modelo')
plt.ylabel('Valor del coeficiente (escala estandarizada)')
plt.tight_layout()
plt.show()


In [ ]:


# ## 9. Validación Cruzada Final del Mejor Modelo

# TODO: Realiza validación cruzada con 10 folds del mejor modelo
# Reporta la media y desviación estándar del RMSE
# Tu código aquí

def rmse_cv_estimator(estimator, X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    # pipeline manual de escalado
    rmses = []
    for tr_idx, te_idx in kf.split(X):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
        sc = StandardScaler()
        X_tr_s = sc.fit_transform(X_tr)
        X_te_s = sc.transform(X_te)
        est = estimator
        est.fit(X_tr_s, y_tr)
        y_hat = est.predict(X_te_s)
        rmses.append(np.sqrt(mean_squared_error(y_te, y_hat)))
    return np.mean(rmses), np.std(rmses)

if best_model_name == 'Linear':
    est = LinearRegression()
elif best_model_name == 'Ridge':
    est = Ridge(alpha=best_alpha_ridge, random_state=42)
else:
    est = Lasso(alpha=lasso_cv.alpha_, random_state=42)

rmse_mean_10cv, rmse_std_10cv = rmse_cv_estimator(est, X, y, n_splits=10)
print(f"Validación cruzada 10-fold ({best_model_name}) -> RMSE medio: {rmse_mean_10cv:.4f} +/- {rmse_std_10cv:.4f}")



In [ ]:

# ## 10. Conclusiones y Preguntas de Reflexión
#
# 1. **¿Cuál modelo tuvo el mejor desempeño? ¿Por qué crees que fue así?**  
#    - Respuesta sugerida: El mejor por RMSE en test fue `{best_model_name}`. Regularizaciones (Ridge/Lasso) suelen mejorar generalización al reducir varianza/overfitting respecto a la regresión lineal pura.
#
# 2. **¿Qué características son las más importantes para predecir la calidad del vino?**  
#    - Respuesta sugerida: En general, *alcohol* (positiva) y *volatile acidity* (negativa) tienden a dominar; *sulphates* y *citric acid* también aportan.
#
# 3. **¿Observas señales de sobreajuste en algún modelo? ¿Cómo lo identificaste?**  
#    - Respuesta sugerida: Comparando R² (train vs test) y RMSE (train vs test). Una brecha grande indicaría sobreajuste.
#
# 4. **¿Cómo cambió el rendimiento de Ridge y Lasso con diferentes valores de alpha?**  
#    - Respuesta sugerida: Con alpha pequeño ≈ modelo lineal; al aumentar alpha, los coeficientes se encogen, sube el sesgo y baja la varianza; hay un punto óptimo.
#
# 5. **¿Qué ventajas observaste al usar validación cruzada para seleccionar hiperparámetros?**  
#    - Respuesta sugerida: Usa mejor los datos, reduce varianza de estimación y evita usar el set de prueba para selección.
#
# 6. **Si Lasso eliminó algunas características, ¿crees que esto mejoró o empeoró el modelo? ¿Por qué?**  
#    - Respuesta sugerida: Puede mejorar al reducir ruido y simplificar el modelo; depende del patrón de correlaciones y de alpha óptimo.
#
# 7. **¿Qué otros pasos podrías tomar para mejorar el rendimiento del modelo?**  
#    - Respuesta sugerida: Ingeniería de características (interacciones), modelos no lineales (árboles/GBM), detección de outliers, validación anidada, calibración de hiperparámetros más fina.
